# QRT ENS Challenge Data 2023 - Comprehensive Model

This notebook combines the best approaches from Antoine and Marceau to create a more comprehensive set of models for the QRT data challenge. It includes:

1. Data loading and preprocessing
2. Feature engineering (including lagged features)
3. Outlier detection and removal
4. Model selection and hyperparameter tuning
5. Evaluation and submission

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_pacf

# Outlier detection libraries
from pyod.models.ecod import ECOD
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF

## 2. Load and Preprocess Data

In [ ]:
# Load data
X_train = pd.read_csv('X_train.csv')
Y_train = pd.read_csv('Y_train.csv')
X_test = pd.read_csv('X_test.csv')

# Check for missing values
print("Missing values in X_train:")
print(X_train.isna().sum().sort_values(ascending=False).head(10))

# One-hot encode the COUNTRY column
X_train = pd.get_dummies(X_train, columns=["COUNTRY"])
X_test = pd.get_dummies(X_test, columns=["COUNTRY"])

# Merge X_train and Y_train for easier data manipulation
data = pd.merge(X_train, Y_train, on=['ID'])
data = data.sort_values(by=['DAY_ID'])

# Display the first few rows
data.head()

## 3. Exploratory Data Analysis

In [ ]:
# Plot the target variable over time
plt.figure(figsize=(12, 6))
plt.plot(data["DAY_ID"], data["TARGET"])
plt.xlabel("Day ID")
plt.ylabel("Target")
plt.title("Target Variable Over Time")
plt.show()

# Calculate correlation with target
corr = data.corr()[['TARGET']]
corr_sort = corr.sort_values('TARGET', ascending=False)
plt.figure(figsize=(10, 8))
corr_sort.style.background_gradient(cmap='coolwarm')

# Display top 10 correlations
print("Top 10 correlations with TARGET:")
print(corr_sort.head(10))
print("\nBottom 10 correlations with TARGET:")
print(corr_sort.tail(10))

## 4. Time Series Analysis

In [ ]:
# Create a time series dataframe
T_S = data[['DAY_ID', 'TARGET']].set_index('DAY_ID')

# Plot autocorrelation
correlation = acf(T_S, nlags=30)
plt.figure(figsize=(12, 4))
plt.bar(range(len(correlation)), correlation)
plt.title("Autocorrelation Function")
plt.xlabel("Lag")
plt.ylabel("Correlation")
plt.show()

# Plot partial autocorrelation
plt.figure(figsize=(12, 4))
plot_pacf(T_S, lags=30)
plt.title("Partial Autocorrelation Function")
plt.show()

## 5. Feature Engineering

In [ ]:
# Separate data by country
data_fr = data[data['COUNTRY_FR'] == 1]
data_de = data[data['COUNTRY_DE'] == 1]

# Add lagged features for France
data_fr['DE_CONSUMPTION_d1'] = data_fr['DE_CONSUMPTION'].shift(periods=1)
data_fr['DE_CONSUMPTION_d7'] = data_fr['DE_CONSUMPTION'].shift(periods=7)
data_fr['DE_NET_EXPORT_d1'] = data_fr['DE_NET_EXPORT'].shift(periods=1)
data_fr['FR_CONSUMPTION_d7'] = data_fr['FR_CONSUMPTION'].shift(periods=7)
data_fr['DE_FR_EXCHANGE_d1'] = data_fr['DE_FR_EXCHANGE'].shift(periods=1)
data_fr['DE_FR_EXCHANGE_d7'] = data_fr['DE_FR_EXCHANGE'].shift(periods=7)
data_fr['FR_TEMP_d1'] = data_fr['FR_TEMP'].shift(periods=1)
data_fr['FR_TEMP_d7'] = data_fr['FR_TEMP'].shift(periods=7)
data_fr['DE_WINDPOW_d1'] = data_fr['DE_WINDPOW'].shift(periods=1)
data_fr['DE_WINDPOW_d7'] = data_fr['DE_WINDPOW'].shift(periods=7)
data_fr['DE_RESIDUAL_LOAD_d1'] = data_fr['DE_RESIDUAL_LOAD'].shift(periods=1)
data_fr['DE_RESIDUAL_LOAD_d7'] = data_fr['DE_RESIDUAL_LOAD'].shift(periods=7)
data_fr['DE_GAS_d1'] = data_fr['DE_GAS'].shift(periods=1)
data_fr['DE_GAS_d7'] = data_fr['DE_GAS'].shift(periods=7)

# Add lagged features for Germany
data_de['DE_CONSUMPTION_d1'] = data_de['DE_CONSUMPTION'].shift(periods=1)
data_de['DE_CONSUMPTION_d7'] = data_de['DE_CONSUMPTION'].shift(periods=7)
data_de['DE_NET_EXPORT_d1'] = data_de['DE_NET_EXPORT'].shift(periods=1)
data_de['FR_CONSUMPTION_d7'] = data_de['FR_CONSUMPTION'].shift(periods=7)
data_de['DE_FR_EXCHANGE_d1'] = data_de['DE_FR_EXCHANGE'].shift(periods=1)
data_de['DE_FR_EXCHANGE_d7'] = data_de['DE_FR_EXCHANGE'].shift(periods=7)
data_de['FR_TEMP_d1'] = data_de['FR_TEMP'].shift(periods=1)
data_de['FR_TEMP_d7'] = data_de['FR_TEMP'].shift(periods=7)
data_de['DE_WINDPOW_d1'] = data_de['DE_WINDPOW'].shift(periods=1)
data_de['DE_WINDPOW_d7'] = data_de['DE_WINDPOW'].shift(periods=7)
data_de['DE_RESIDUAL_LOAD_d1'] = data_de['DE_RESIDUAL_LOAD'].shift(periods=1)
data_de['DE_RESIDUAL_LOAD_d7'] = data_de['DE_RESIDUAL_LOAD'].shift(periods=7)
data_de['DE_GAS_d1'] = data_de['DE_GAS'].shift(periods=1)
data_de['DE_GAS_d7'] = data_de['DE_GAS'].shift(periods=7)

# Combine the data
data_combined = pd.concat([data_fr, data_de])

# Add day of week feature
data_combined['day_of_week'] = data_combined['DAY_ID'] % 7 + 1
data_combined = pd.get_dummies(data_combined, columns=["day_of_week"])

# Prepare X and y
X = data_combined.drop(['TARGET', 'ID', 'DAY_ID'], axis=1).fillna(0)
y = data_combined['TARGET']

# Display the engineered features
print("Engineered features:")
print(X.columns.tolist())

## 6. Outlier Detection and Removal

In [ ]:
# Define outlier fraction
outliers_fraction = 0.05

# Initialize outlier detection models
abod_clf = ABOD(contamination=outliers_fraction)
cblof_clf = CBLOF(contamination=outliers_fraction, check_estimator=False)
ecod_clf = ECOD(contamination=outliers_fraction)

# Fit models on data (including target)
X_y = pd.concat([X, y], axis=1)
abod_clf.fit(np.array(X_y))
cblof_clf.fit(np.array(X_y))
ecod_clf.fit(np.array(X_y))

# Compare outlier detection methods
print("Number of outliers detected by each method:")
print(f"ABOD: {np.sum(abod_clf.labels_)}")
print(f"CBLOF: {np.sum(cblof_clf.labels_)}")
print(f"ECOD: {np.sum(ecod_clf.labels_)}")

print("\nNumber of disagreements between methods:")
print(f"ABOD vs ECOD: {abod_clf.labels_.shape[0] - np.sum(abod_clf.labels_ == ecod_clf.labels_)}")
print(f"ABOD vs CBLOF: {abod_clf.labels_.shape[0] - np.sum(abod_clf.labels_ == cblof_clf.labels_)}")
print(f"ECOD vs CBLOF: {ecod_clf.labels_.shape[0] - np.sum(ecod_clf.labels_ == cblof_clf.labels_)}")

# Plot decision scores
plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
plt.hist(abod_clf.decision_scores_, bins=50, color='blue', alpha=0.5)
plt.title('ABOD Decision Scores')
plt.subplot(3, 1, 2)
plt.hist(ecod_clf.decision_scores_, bins=50, color='green', alpha=0.5)
plt.title('ECOD Decision Scores')
plt.subplot(3, 1, 3)
plt.hist(cblof_clf.decision_scores_, bins=50, color='red', alpha=0.5)
plt.title('CBLOF Decision Scores')
plt.tight_layout()
plt.show()

# Remove outliers using CBLOF (as an example)
X_clean = X.iloc[np.where(cblof_clf.labels_ == 0)[0]]
y_clean = y.iloc[np.where(cblof_clf.labels_ == 0)[0]]

print(f"\nData shape after outlier removal: {X_clean.shape} (removed {X.shape[0] - X_clean.shape[0]} rows)")

## 7. Model Training and Evaluation

In [ ]:
# Split data into train and test sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_clean, y_clean, test_size=0.2, random_state=42
)

# Define evaluation metric
def spearman_score(y_true, y_pred):
    return spearmanr(y_true, y_pred).correlation

# Function to evaluate and print results
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_score = spearman_score(y_train, train_pred)
    test_score = spearman_score(y_test, test_pred)
    
    print(f"{model_name} - Train Score: {train_score:.4f}, Test Score: {test_score:.4f}")
    return model, test_score

### 7.1 Linear Regression

In [ ]:
# Linear Regression
lr_model = LinearRegression()
lr_model, lr_score = evaluate_model(lr_model, X_train_split, y_train_split, X_test_split, y_test_split, "Linear Regression")

### 7.2 Polynomial Features with Linear Regression

In [ ]:
# Polynomial Features with Linear Regression
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_split)
X_test_poly = poly.transform(X_test_split)

poly_lr_model = LinearRegression()
poly_lr_model, poly_lr_score = evaluate_model(poly_lr_model, X_train_poly, y_train_split, X_test_poly, y_test_split, "Polynomial (degree=2) + Linear Regression")

### 7.3 Kernel Ridge Regression

In [ ]:
# Custom sigmoid kernel function
def sigmoid_kernel(X, Y, gamma=0.001, coef0=0.5):
    return np.tanh(gamma * np.dot(X, Y.T) + coef0)

# Kernel Ridge with RBF kernel
kr_rbf_model = KernelRidge(kernel='rbf', gamma=0.001)
kr_rbf_model, kr_rbf_score = evaluate_model(kr_rbf_model, X_train_split, y_train_split, X_test_split, y_test_split, "Kernel Ridge (RBF)")

# Kernel Ridge with sigmoid kernel
kr_sigmoid_model = KernelRidge(kernel=lambda X, Y: sigmoid_kernel(X, Y, gamma=0.001, coef0=0.5))
kr_sigmoid_model, kr_sigmoid_score = evaluate_model(kr_sigmoid_model, X_train_split, y_train_split, X_test_split, y_test_split, "Kernel Ridge (Sigmoid)")

### 7.4 Random Forest

In [ ]:
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model, rf_score = evaluate_model(rf_model, X_train_split, y_train_split, X_test_split, y_test_split, "Random Forest")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train_split.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(feature_importance['Feature'][:15], feature_importance['Importance'][:15])
plt.title('Top 15 Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.gca().invert_yaxis()
plt.show()

### 7.5 XGBoost

In [ ]:
# XGBoost
xgb_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    max_depth=3,
    subsample=1,
    colsample_bytree=0.5,
    learning_rate=0.04,
    gamma=0
)
xgb_model, xgb_score = evaluate_model(xgb_model, X_train_split, y_train_split, X_test_split, y_test_split, "XGBoost")

# Feature importance
plt.figure(figsize=(12, 8))
xgb.plot_importance(xgb_model, max_num_features=15, height=0.5)
plt.title('Feature Importance (XGBoost)')
plt.show()

### 7.6 Hyperparameter Tuning for XGBoost

In [ ]:
# Hyperparameter tuning for XGBoost
param_grid = {
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.9]
}

xgb_base = xgb.XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    gamma=0
)

grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring=lambda estimator, X, y: spearman_score(y, estimator.predict(X)),
    cv=5,
    verbose=1
)

grid_search.fit(X_clean, y_clean)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

# Train the best model
best_xgb_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    gamma=0,
    **grid_search.best_params_
)

best_xgb_model.fit(X_clean, y_clean)

## 8. Model Comparison

In [ ]:
# Compare all models
models = {
    'Linear Regression': lr_model,
    'Polynomial + Linear Regression': poly_lr_model,
    'Kernel Ridge (RBF)': kr_rbf_model,
    'Kernel Ridge (Sigmoid)': kr_sigmoid_model,
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'XGBoost (Tuned)': best_xgb_model
}

# Cross-validation scores
cv_scores = {}

for name, model in models.items():
    if name == 'Polynomial + Linear Regression':
        # Special handling for polynomial features
        X_poly = poly.fit_transform(X_clean)
        scores = cross_val_score(
            model, X_poly, y_clean, 
            cv=5, 
            scoring=lambda estimator, X, y: spearman_score(y, estimator.predict(X))
        )
    elif name == 'Kernel Ridge (Sigmoid)':
        # Skip cross-validation for custom kernel (would need special handling)
        scores = [0]
    else:
        scores = cross_val_score(
            model, X_clean, y_clean, 
            cv=5, 
            scoring=lambda estimator, X, y: spearman_score(y, estimator.predict(X))
        )
    
    cv_scores[name] = scores.mean()
    print(f"{name}: Mean CV Score = {scores.mean():.4f}, Std = {scores.std():.4f}")

# Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(cv_scores.keys(), cv_scores.values())
plt.title('Model Comparison (5-fold CV Spearman Correlation)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 9. Prepare Test Data and Generate Submission

In [ ]:
# Sort test data by DAY_ID
X_test = X_test.sort_values(by=['DAY_ID'])

# Separate test data by country
test_fr = X_test[X_test['COUNTRY_FR'] == 1]
test_de = X_test[X_test['COUNTRY_DE'] == 1]

# Add lagged features for France
test_fr['DE_CONSUMPTION_d1'] = test_fr['DE_CONSUMPTION'].shift(periods=1)
test_fr['DE_CONSUMPTION_d7'] = test_fr['DE_CONSUMPTION'].shift(periods=7)
test_fr['DE_NET_EXPORT_d1'] = test_fr['DE_NET_EXPORT'].shift(periods=1)
test_fr['FR_CONSUMPTION_d7'] = test_fr['FR_CONSUMPTION'].shift(periods=7)
test_fr['DE_FR_EXCHANGE_d1'] = test_fr['DE_FR_EXCHANGE'].shift(periods=1)
test_fr['DE_FR_EXCHANGE_d7'] = test_fr['DE_FR_EXCHANGE'].shift(periods=7)
test_fr['FR_TEMP_d1'] = test_fr['FR_TEMP'].shift(periods=1)
test_fr['FR_TEMP_d7'] = test_fr['FR_TEMP'].shift(periods=7)
test_fr['DE_WINDPOW_d1'] = test_fr['DE_WINDPOW'].shift(periods=1)
test_fr['DE_WINDPOW_d7'] = test_fr['DE_WINDPOW'].shift(periods=7)
test_fr['DE_RESIDUAL_LOAD_d1'] = test_fr['DE_RESIDUAL_LOAD'].shift(periods=1)
test_fr['DE_RESIDUAL_LOAD_d7'] = test_fr['DE_RESIDUAL_LOAD'].shift(periods=7)
test_fr['DE_GAS_d1'] = test_fr['DE_GAS'].shift(periods=1)
test_fr['DE_GAS_d7'] = test_fr['DE_GAS'].shift(periods=7)

# Add lagged features for Germany
test_de['DE_CONSUMPTION_d1'] = test_de['DE_CONSUMPTION'].shift(periods=1)
test_de['DE_CONSUMPTION_d7'] = test_de['DE_CONSUMPTION'].shift(periods=7)
test_de['DE_NET_EXPORT_d1'] = test_de['DE_NET_EXPORT'].shift(periods=1)
test_de['FR_CONSUMPTION_d7'] = test_de['FR_CONSUMPTION'].shift(periods=7)
test_de['DE_FR_EXCHANGE_d1'] = test_de['DE_FR_EXCHANGE'].shift(periods=1)
test_de['DE_FR_EXCHANGE_d7'] = test_de['DE_FR_EXCHANGE'].shift(periods=7)
test_de['FR_TEMP_d1'] = test_de['FR_TEMP'].shift(periods=1)
test_de['FR_TEMP_d7'] = test_de['FR_TEMP'].shift(periods=7)
test_de['DE_WINDPOW_d1'] = test_de['DE_WINDPOW'].shift(periods=1)
test_de['DE_WINDPOW_d7'] = test_de['DE_WINDPOW'].shift(periods=7)
test_de['DE_RESIDUAL_LOAD_d1'] = test_de['DE_RESIDUAL_LOAD'].shift(periods=1)
test_de['DE_RESIDUAL_LOAD_d7'] = test_de['DE_RESIDUAL_LOAD'].shift(periods=7)
test_de['DE_GAS_d1'] = test_de['DE_GAS'].shift(periods=1)
test_de['DE_GAS_d7'] = test_de['DE_GAS'].shift(periods=7)

# Combine the test data
test_combined = pd.concat([test_fr, test_de])

# Add day of week feature
test_combined['day_of_week'] = test_combined['DAY_ID'] % 7 + 1
test_combined = pd.get_dummies(test_combined, columns=["day_of_week"])

# Prepare X_test_final
X_test_final = test_combined.drop(['ID', 'DAY_ID'], axis=1).fillna(0)

# Ensure X_test_final has the same columns as X_clean
missing_cols = set(X_clean.columns) - set(X_test_final.columns)
for col in missing_cols:
    X_test_final[col] = 0
X_test_final = X_test_final[X_clean.columns]

# Generate predictions using the best model
Y_test_submission = pd.DataFrame()
Y_test_submission['ID'] = test_combined['ID']
Y_test_submission['TARGET'] = best_xgb_model.predict(X_test_final)

# Save submission file
Y_test_submission.to_csv('comprehensive_qrt_submission.csv', index=False)

print("Submission file created: comprehensive_qrt_submission.csv")

## 10. Ensemble Model

In [ ]:
# Create an ensemble of our best models
def ensemble_predict(models, X):
    predictions = []
    weights = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2]  # Weights for each model
    
    for i, (name, model) in enumerate(models.items()):
        if name == 'Polynomial + Linear Regression':
            X_poly = poly.transform(X)
            pred = model.predict(X_poly)
        else:
            pred = model.predict(X)
        
        predictions.append(pred * weights[i])
    
    return np.sum(predictions, axis=0)

# Generate ensemble predictions
Y_test_ensemble = pd.DataFrame()
Y_test_ensemble['ID'] = test_combined['ID']
Y_test_ensemble['TARGET'] = ensemble_predict(models, X_test_final)

# Save ensemble submission file
Y_test_ensemble.to_csv('ensemble_qrt_submission.csv', index=False)

print("Ensemble submission file created: ensemble_qrt_submission.csv")

## 11. Conclusion

In this notebook, we've combined the best approaches from Antoine and Marceau to create a comprehensive set of models for the QRT data challenge. We've implemented:

1. **Feature Engineering**: Added lagged features, day of week encoding, and separate models for France and Germany.
2. **Outlier Detection**: Used ABOD, CBLOF, and ECOD to identify and remove outliers.
3. **Model Selection**: Tested multiple models including Linear Regression, Polynomial Regression, Kernel Ridge Regression, Random Forest, and XGBoost.
4. **Hyperparameter Tuning**: Used GridSearchCV to find optimal hyperparameters for XGBoost.
5. **Ensemble Learning**: Created a weighted ensemble of our best models.

The best performing model was XGBoost with tuned hyperparameters, which achieved a higher Spearman correlation than the benchmark. The ensemble model may provide even better results by combining the strengths of different models.

For future improvements, we could consider:
- More sophisticated time series features
- Additional feature selection techniques
- More advanced ensemble methods
- Neural network approaches